# BC 4
## Flat ensemble of tiny networks
## Input = whole image

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas as pd
import os
import random as rn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
SEED = 321
rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
NUM_TN = 30

#dataset specific parameters
NUM_CLASS = 5
SCALE = 32

In [4]:
data = "../input/flowers-recognition/flowers/"
folders = os.listdir(data)

In [5]:
image_names = []
train_labels = []
train_images = []

size = 32,32

for folder in folders:
    for file in os.listdir(os.path.join(data,folder)):
        if file.endswith("jpg"):
            image_names.append(os.path.join(data,folder,file))
            train_labels.append(folder)
            img = cv2.imread(os.path.join(data,folder,file))
            im = cv2.resize(img,size)
            train_images.append(im)
        else:
            continue

In [6]:
train = np.array(train_images)
train = train.astype('float32')
train.shape

(4323, 32, 32, 3)

In [7]:
# Train, Val, Test split = 0.8, 0.1, 0.1 of the dataset
X_train,X_val,y_train,y_val = train_test_split(train,train_labels, test_size = 0.2)
X_val,X_test,y_val,y_test = train_test_split(X_val,y_val, test_size = 0.5)

## Data Pre-Processing

In [8]:
#Generate subset of train image
X_train_subset_list = []
y_train_subset_list = []
train_zipped = zip(X_train, y_train)
train_zipped = np.array(tuple(train_zipped))

for i in range(NUM_TN):
  train_zipped_subset = rn.choices(train_zipped, k=int(len(train_zipped)*60/100))
  X_train_subset = []
  y_train_subset = []
  for a in train_zipped_subset:
    X_train_subset.append(a[0])
    y_train_subset.append(a[1])
  X_train_subset = np.array(X_train_subset)
  y_train_subset = np.array(y_train_subset)

  X_train_subset_list.append(X_train_subset)
  y_train_subset_list.append(y_train_subset)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [9]:
#normalizing dataset
for i in range(NUM_TN):
    X_train_subset_list[i] = X_train_subset_list[i]/255

X_test = X_test/255
X_val = X_val/255

In [10]:
#transform to vector
for i in range(NUM_TN):
    X_train_subset_list[i] = X_train_subset_list[i].reshape((-1, SCALE*SCALE*3))

X_val = X_val.reshape((-1, SCALE*SCALE*3))
X_test = X_test.reshape((-1, SCALE*SCALE*3))

In [11]:
#one-hot encoding
for i in range(NUM_TN):
    y_train_subset_list[i] = pd.get_dummies(y_train_subset_list[i])
    y_train_subset_list[i] = y_train_subset_list[i].values.argmax(1)
    y_train_subset_list[i] = to_categorical(y_train_subset_list[i])
    

y_val_one_hot = pd.get_dummies(y_val)
y_val_one_hot = y_val_one_hot.values.argmax(1)
y_val_one_hot = to_categorical(y_val_one_hot)

y_test_one_hot = pd.get_dummies(y_test)
y_test_one_hot = y_test_one_hot.values.argmax(1)
y_test_one_hot = to_categorical(y_test_one_hot)

## Model

In [12]:
#build ANN model
ensemble = []
for i in range(NUM_TN):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=SCALE*SCALE*3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(NUM_CLASS, activation = 'softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    ensemble.append(model)

In [13]:
#train model
history = []
for i in range(NUM_TN):
    print()
    print('Model %d' %i)
    hist = ensemble[i].fit(X_train_subset_list[i], y_train_subset_list[i], validation_data=(X_val, y_val_one_hot), epochs=5, batch_size=128)
    history.append(hist)


Model 0
Epoch 1/5
17/17 [==============================] - 2s 35ms/step - loss: 1.6261 - accuracy: 0.2690 - val_loss: 1.4881 - val_accuracy: 0.3796
Epoch 2/5
17/17 [==============================] - 0s 6ms/step - loss: 1.4344 - accuracy: 0.3764 - val_loss: 1.3779 - val_accuracy: 0.4074
Epoch 3/5
17/17 [==============================] - 0s 6ms/step - loss: 1.3157 - accuracy: 0.4420 - val_loss: 1.2614 - val_accuracy: 0.4444
Epoch 4/5
17/17 [==============================] - 0s 6ms/step - loss: 1.2362 - accuracy: 0.4661 - val_loss: 1.3057 - val_accuracy: 0.4375
Epoch 5/5
17/17 [==============================] - 0s 6ms/step - loss: 1.2315 - accuracy: 0.4711 - val_loss: 1.2651 - val_accuracy: 0.4514

Model 1
Epoch 1/5
17/17 [==============================] - 1s 14ms/step - loss: 1.6431 - accuracy: 0.2451 - val_loss: 1.4429 - val_accuracy: 0.3611
Epoch 2/5
17/17 [==============================] - 0s 6ms/step - loss: 1.4269 - accuracy: 0.3592 - val_loss: 1.3505 - val_accuracy: 0.4491
Epoch 3

## Model Evaluation

In [14]:
# Predict on test image
y_pred_list = []
for i in range(NUM_TN):
    y_pred = ensemble[i].predict(X_test)
    y_pred_list.append(y_pred)

# Ensemble voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.argmax(np.sum(y_pred_list, axis=0), axis=1)

In [15]:
y_test_label_encoded = pd.get_dummies(y_test)
y_test_label_encoded = y_test_label_encoded.values.argmax(1)

In [16]:
accuracy = accuracy_score(y_test_label_encoded, y_pred_list)
accuracy

0.4457274826789838